# PGR Data Loader - User Guide

## Purpose
This notebook helps you load master data into the PGR (Public Grievance Redressal) system using Excel templates.

## What You'll Do
1. Fill Excel templates with your data (cities, departments, complaint types, etc.)
2. Run this notebook to automatically upload everything to the system
3. Check results to see what was uploaded successfully

## Prerequisites
Before you start, make sure you have:
- Access credentials (username/password)
- Excel templates filled with your data
- Excel file in the `templates/` folder

## Excel Templates - Fill in This Order

| Order | Template | What to Fill |
|-------|----------|--------------|
| 1 | `ExcelSheets_PGR_Master_Data_UNIFIED.xlsx` | State & Cities information , Geographic boundaries, Government departments, Job titles/positions, Complaint types,  Language translations (Optional) | REQUIRED |
| 2 | `GoogleSheets_PGR_Master_Data_UNIFIED.xlsx` | State & Cities information , Geographic boundaries, Government departments, Job titles/positions, Complaint types,  Language translations (Optional) |

### How to Access Templates

**Option 1: Using Jupyter File Browser (Recommended)**
1. Click on "Jupyter" logo (top left) to go to file browser
2. Navigate to: `pgr DataLoader` → `templates/`
3. Click on any `.xlsx` file to download it
4. Fill the template

**Option 2: Using File System**
- Go to folder: `/Users/salaudeenn/Documents/urban/pgr/pgr DataLoader/templates/`
- Double-click any template to open in Excel
- Fill and save

**Option 3: Run the cell below to see templates location**

---

## How to Use This Notebook

### Step 1: Access and fill templates
- Use one of the methods above to open templates
- Fill each template with your data

### Step 2: Run cells in order from top to bottom
- Click on each cell and press **Shift + Enter** to run it

### Step 3: Check for success or error messages
- Look for "Created" (success) or "Failed" (error) messages

### Step 4: If you see errors
- Read the error message
- Fix the Excel template
- Re-run that cell

---

## Important Notes

1. Do NOT skip cells - Run them in order
2. Fill templates BEFORE running - The notebook reads from Excel files
3. Check your data - Make sure codes are unique and names are correct
4. Save Excel files - Always save after editing
5. Follow the order - Tenant → Boundary → Department → Designation → Workflow → Localisation

---

## Need Help?

If you see errors, check:
1. Is the Excel file in the `templates/` folder?
2. Did you fill all required columns?
3. Are codes unique (no duplicates)?
4. Did you save the Excel file?
5. Did you follow the correct order?


---

Let's begin.

In [1]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import json
import requests
import time 
import os
from datetime import datetime
from typing import Dict,List,Any
import warnings
warnings.filterwarnings('ignore')

# Import the unified loader module
from unified_loader import UnifiedExcelReader, APIUploader, clean_nans
import json
from collections import defaultdict


print("Package are Loaded")

Package are Loaded


In [3]:
# Template Download Links
# Click any link below to download the template file

from IPython.display import display, HTML, FileLink
import os

templates_path = "templates"

print("="*70)
print("EXCEL TEMPLATE DOWNLOAD LINKS")
print("="*70)
print("\nClick any link below to download the template:\n")

# Template files in order
template_files = [
    ("ExcelSheets_PGR_Master_Data_UNIFIED.xlsx", "State & Cities information , Geographic boundaries, Government departments, Job titles/positions, Complaint types,  Language translations (Optional)"),
    ("GoogleSheets_PGR_Master_Data_UNIFIED.xlsx", "State & Cities information , Geographic boundaries, Government departments, Job titles/positions, Complaint types,  Language translations (Optional)"),
    # ("4_Boundaries_Template.xlsx", "Geographic boundaries"),
    # ("2_Departments_Template.xlsx", "Government departments"),
    # ("3_Designations_Template.xlsx", "Job titles/positions"),
    # ("7_Workflow_Template.xlsx", "Complaint processing flow"),
    # ("6_Localization_Template.xlsx", "Language translations (Optional)"),
    # ("1_ComplaintTypes_Template.xlsx", "Complaint types")
]

# Create clickable download links
html_links = []
for i, (filename, description) in enumerate(template_files, 1):
    filepath = os.path.join(templates_path, filename)
    if os.path.exists(filepath):
        html_links.append(
            f'<div style="margin: 10px 0;">'
            f'<b>{i}.</b> '
            f'<a href="{filepath}" download="{filename}" '
            f'style="font-size: 14px; color: #0066cc; text-decoration: none;">'
            f'{filename}</a> '
            f'<span style="color: #666;">- {description}</span>'
            f'</div>'
        )
    else:
        html_links.append(
            f'<div style="margin: 10px 0;">'
            f'<b>{i}.</b> {filename} <span style="color: red;">(Not found)</span>'
            f'</div>'
        )

# Display as HTML with styling
html_content = f"""
<div style="border: 1px solid #ddd; padding: 15px; border-radius: 5px; background-color: #f9f9f9;">
    <h3 style="margin-top: 0;">Template Files:</h3>
    {''.join(html_links)}
    <hr style="margin: 20px 0;">
    <p><b>Instructions:</b></p>
    <ol>
        <li>Click any template link above to download it</li>
        <li>Open the file in Microsoft Excel or Google Sheets</li>
        <li>Fill in your data following the column headers</li>
    </ol>
</div>
"""

display(HTML(html_content))

# print("\n" + "="*70)
# print("Alternative: Use Jupyter file browser")
# print("="*70)
# print("1. Click 'Jupyter' logo (top-left)")
# print("2. Navigate to: pgr DataLoader → templates/")
# print("3. Right-click any .xlsx file and select 'Download'")
# print("="*70)

EXCEL TEMPLATE DOWNLOAD LINKS

Click any link below to download the template:



## STEP 1: CONFIGURATION

In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import pandas as pd

# Initialize CONFIG
CONFIG = {}
CONFIG_SET = False
EXCEL_FILES = {}

print("="*70)
print("        ⚙️  PGR DATA LOADER - CONFIGURATION SETUP")
print("="*70)
print()

# Configuration widgets
base_url = widgets.Text(
    value='https://unified-qa.digit.org',
    description='🌐 Base URL:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

user_type = widgets.Dropdown(
    options=['EMPLOYEE', 'CITIZEN'],
    value='EMPLOYEE',
    description='👔 User Type:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

tenant_id = widgets.Text(
    value='',
    description='🏛️ State Tenant:',
    placeholder='e.g., pg',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

target_tenant_id = widgets.Text(
    value='',
    description='🏙️ City Tenant:',
    placeholder='e.g., pg.citya',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

save_btn = widgets.Button(
    description='✅ Save Configuration',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

output1 = widgets.Output()

# Individual file upload widgets for each template
file_uploads = {
     'pgr_master_data': widgets.FileUpload(
        accept='.xlsx,.xls',
        multiple=False,
        description='',
        layout=widgets.Layout(width='70%')
    ),
    'tenant': widgets.FileUpload(
        accept='.xlsx,.xls',
        multiple=False,
        description='',
        layout=widgets.Layout(width='70%')
    ),
    'departments': widgets.FileUpload(
        accept='.xlsx,.xls',
        multiple=False,
        description='',
        layout=widgets.Layout(width='70%')
    ),
    'designations': widgets.FileUpload(
        accept='.xlsx,.xls',
        multiple=False,
        description='',
        layout=widgets.Layout(width='70%')
    ),
    'complaint_types': widgets.FileUpload(
        accept='.xlsx,.xls',
        multiple=False,
        description='',
        layout=widgets.Layout(width='70%')
    ),
    'boundaries': widgets.FileUpload(
        accept='.xlsx,.xls',
        multiple=False,
        description='',
        layout=widgets.Layout(width='70%')
    ),
    'localization': widgets.FileUpload(
        accept='.xlsx,.xls',
        multiple=False,
        description='',
        layout=widgets.Layout(width='70%')
    ),
    'workflow': widgets.FileUpload(
        accept='.xlsx,.xls',
        multiple=False,
        description='',
        layout=widgets.Layout(width='70%')
    )
}

# Mapping of each template key to standardized filename and schema
template_config = {
    'pgr_master_data': {
        'filename': 'PGR_Master_Data_UNIFIED.xlsx',
        'schema': 'pgr_master_data_unified_schema.yaml',
        'display_name': 'Tenant'
    },
    # 'tenant': {
    #     'filename': '0_Tenant_Template.xlsx',
    #     'schema': 'tenant_schema.yaml',
    #     'display_name': 'Tenant'
    # },
    # 'departments': {
    #     'filename': '2_Departments_Template.xlsx',
    #     'schema': 'departments_schema.yaml',
    #     'display_name': 'Departments'
    # },
    # 'designations': {
    #     'filename': '3_Designations_Template.xlsx',
    #     'schema': 'designations_schema.yaml',
    #     'display_name': 'Designations'
    # },
    # 'complaint_types': {
    #     'filename': '1_ComplaintTypes_Template.xlsx',
    #     'schema': 'complaint_types_schema.yaml',
    #     'display_name': 'Complaint Types'
    # },
    # 'boundaries': {
    #     'filename': '4_Boundaries_Template.xlsx',
    #     'schema': 'boundaries_schema.yaml',
    #     'display_name': 'Boundaries'
    # },
    # 'workflow': {
    #     'filename': '7_Workflow_Template.xlsx',
    #     'schema': 'workflow_schema.yaml',
    #     'display_name': 'Workflow'
    # },
    # 'localization': {
    #     'filename': '7_Workflow_Template_Production.xlsx',
    #     'schema': 'localization_schema.yaml',
    #     'display_name': 'Localization'
    # }
}

template_labels = {
    'pgr_master_data': 'PGR_Master_Data_UNIFIED.xlsx',
    # 'tenant': '🏢 Tenant Template (0_Tenant_Template.xlsx)',
    # 'complaint_types': '📝 Complaint Types Template (1_ComplaintTypes_Template.xlsx)',
    # 'departments': '🏛️ Departments Template (2_Departments_Template.xlsx)',
    # 'designations': '💼 Designations Template (3_Designations_Template.xlsx)',
    # 'boundaries': '🗺️ Boundaries Template (4_Boundaries_Template.xlsx)',
    # 'workflow': '⚙️ Workflow Template (7_Workflow_Template.xlsx)',
    # 'localization': '🌐 Localization Template (7_Workflow_Template_Production.xlsx)'
}

# Create individual upload rows (UI fields)
upload_rows = []
# for key in ['pgr_master_data','tenant', 'complaint_types', 'departments', 'designations', 'boundaries', 'workflow', 'localization']:
for key in ['pgr_master_data']:
    row = widgets.HBox([
        widgets.HTML(f"<div style='width:450px; padding:5px'>{template_labels[key]}</div>"),
        file_uploads[key]
    ])
    upload_rows.append(row)

upload_all_btn = widgets.Button(
    description='⬆️ Validate & Upload All Selected Files',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px')
)

output2 = widgets.Output()

# Config box
config_box = widgets.VBox([
    widgets.HTML("<h3>📋 Step 1: Configuration</h3>"),
    base_url,
    user_type,
    tenant_id,
    target_tenant_id,
    save_btn,
    output1
])

upload_box = widgets.VBox([
    widgets.HTML("<h3>📁 Step 2: Upload Excel Templates</h3>"),
    widgets.HTML("<p><i>Select individual template files below. Files will be validated before saving to ensure data quality.</i></p>"),
    *upload_rows,
    widgets.HTML("<br>"),
    upload_all_btn,
    output2
])

upload_box.layout.visibility = 'hidden'
upload_box.layout.display = 'none'

# Event handlers
def on_save(b):
    global CONFIG, CONFIG_SET

    with output1:
        clear_output()
        errors = []
        if not base_url.value.strip():
            errors.append("❌ Base URL required")
        if not tenant_id.value.strip():
            errors.append("❌ State Tenant ID required")
        if not target_tenant_id.value.strip():
            errors.append("❌ City Tenant ID required")
        if errors:
            for err in errors:
                print(err)
            return
        CONFIG = {
            'base_url': base_url.value.strip(),
            'user_type': user_type.value,
            'tenant_id': tenant_id.value.strip(),
            'target_tenant_id': target_tenant_id.value.strip()
        }
        CONFIG_SET = True
        print("="*70)
        print("  ✅ CONFIGURATION SAVED!")
        print("="*70)
        print(f"\n📊 Summary:")
        print(f"   • API: {CONFIG['base_url']}")
        print(f"   • User Type: {CONFIG['user_type']}")
        print(f"   • State Tenant: {CONFIG['tenant_id']}")
        print(f"   • City Tenant: {CONFIG['target_tenant_id']}")
        print("="*70)
        # Show upload box
        config_box.layout.visibility = 'hidden'
        config_box.layout.display = 'none'
        upload_box.layout.visibility = 'visible'
        upload_box.layout.display = 'block'

def validate_excel_file(file_content, filename, schema_file, display_name):
    """
    Validate Excel file content before saving
    Returns tuple: (is_valid, error_messages)
    """
    try:
        # Load ExcelValidator
        import sys
        sys.path.insert(0, os.path.abspath('.'))
        from excel_validator import ExcelValidator
        
        # Create temp file for validation
        temp_dir = 'temp_validation'
        os.makedirs(temp_dir, exist_ok=True)
        temp_path = os.path.join(temp_dir, filename)
        
        # Write content to temp file
        with open(temp_path, 'wb') as f:
            f.write(file_content)
        
        # Validate
        validator = ExcelValidator(schemas_dir="schemas", templates_dir=temp_dir)
        result = validator.validate_file(filename, schema_file)
        
        # Clean up temp file
        os.remove(temp_path)
        
        if result['valid']:
            return True, []
        else:
            # Format error messages
            error_messages = []
            errors_by_sheet = {}
            
            for error in result['errors']:
                sheet = error.get('sheet', 'Unknown')
                if sheet not in errors_by_sheet:
                    errors_by_sheet[sheet] = []
                errors_by_sheet[sheet].append(error)
            
            for sheet_name, sheet_errors in errors_by_sheet.items():
                error_messages.append(f"\n   Sheet: {sheet_name}")
                for i, error in enumerate(sheet_errors[:3], 1):  # Show first 3 errors per sheet
                    if 'row' in error:
                        error_messages.append(
                            f"      {i}. Row {error['row']}, Column '{error['column']}'"
                        )
                        error_messages.append(f"         Error: {error['message']}")
                    else:
                        msg = error['message']
                        if msg.startswith(f"Sheet '{sheet_name}': "):
                            msg = msg.replace(f"Sheet '{sheet_name}': ", "")
                        error_messages.append(f"      {i}. {msg}")
                
                if len(sheet_errors) > 3:
                    error_messages.append(f"      ... and {len(sheet_errors) - 3} more errors")
            
            return False, error_messages
            
    except ImportError:
        # If validator not available, skip validation
        print(f"   ⚠️  Validator not available for {display_name}, saving without validation")
        return True, []
    except Exception as e:
        # If validation fails for any reason, show warning but allow save
        print(f"   ⚠️  Validation error for {display_name}: {str(e)}")
        return True, []

def on_upload(b):
    global EXCEL_FILES

    with output2:
        clear_output()
        os.makedirs('upload', exist_ok=True)
        
        uploaded_count = 0
        failed_count = 0
        validation_failures = []
        
        print("="*70)
        print("  📋 VALIDATING & UPLOADING FILES")
        print("="*70)
        print()
        
        for key, file_widget in file_uploads.items():
            if file_widget.value:
                uploaded_file = file_widget.value[0]
                content = uploaded_file['content']
                
                config = template_config.get(key)
                if not config:
                    continue
                
                filename = config['filename']
                schema_file = config['schema']
                display_name = config['display_name']
                
                print(f"[{display_name}]")
                print(f"   📄 File: {filename}")
                print(f"   🔍 Validating...", end=" ")
                
                # Validate before saving
                is_valid, errors = validate_excel_file(content, filename, schema_file, display_name)
                
                if is_valid:
                    print("✅ PASSED")
                    # Save file
                    upload_path = os.path.join('upload', filename)
                    with open(upload_path, 'wb') as f:
                        f.write(content)
                    EXCEL_FILES[key] = upload_path
                    uploaded_count += 1
                    print(f"   💾 Saved to: {upload_path}")
                else:
                    print("❌ FAILED")
                    failed_count += 1
                    validation_failures.append({
                        'name': filename,
                        'errors': errors
                    })
                    print("   ⚠️  File NOT saved due to validation errors:")
                    for error in errors:
                        print(error)
                
                print()
        
        if uploaded_count == 0 and failed_count == 0:
            print("❌ No files selected. Please select at least one file.")
            return
        CONFIG['excel_files'] = EXCEL_FILES['pgr_master_data']
        print("="*70)
        print("  📊 UPLOAD SUMMARY")
        print("="*70)
        print(f"\n   ✅ Successfully validated & uploaded: {uploaded_count} file(s)")
        
        if failed_count > 0:
            print(f"   ❌ Validation failed (not saved): {failed_count} file(s)")
            print("\n   📋 Failed Files:")
            for failure in validation_failures:
                print(f"      • {failure['name']}")
            print("\n   💡 Fix the errors in your Excel files and try uploading again.")
        else:
            print("\n   ➡️  All files validated successfully! Ready to proceed.")
            print("   💾 Files saved to 'upload' folder with standardized filenames")
        
        print("="*70)

save_btn.on_click(on_save)
upload_all_btn.on_click(on_upload)

# Display boxes
display(config_box)
display(upload_box)

print("💡 Fill in the details above and click 'Save Configuration'\n")

        ⚙️  PGR DATA LOADER - CONFIGURATION SETUP



💡 Fill in the details above and click 'Save Configuration'



---

## VALIDATE EXCEL TEMPLATES

### What This Does
Validates your Excel templates against schema definitions BEFORE uploading to the server.

### Why Validate?
- Catch errors early (missing columns, wrong formats, duplicates)
- Save time (fix issues before API calls)
- Ensure data quality (check required fields, patterns, references)

### Action Required
Run the cell below to validate all your Excel templates.

### What Gets Checked?
- Required columns exist
- Data types are correct (integers, emails, URLs, etc.)
- Required fields are not empty
- Codes are unique (no duplicates)
- Foreign key references are valid (e.g., Department Codes match)
- Boolean fields use TRUE/FALSE
- Patterns match (e.g., tenant codes, state codes)

### Output
- PASSED: Template is valid, ready to upload
- FAILED: Shows specific errors with row numbers and columns

---

## STEP 2: INITIALIZE READER

In [6]:
# Initialize the Excel reader
reader = UnifiedExcelReader(CONFIG['excel_files'])
uploader = APIUploader()

print("[SUCCESS] Excel reader initialized")
print(f"   File: {CONFIG['excel_files']}")

[SUCCESS] Excel reader initialized
   File: upload/PGR_Master_Data_UNIFIED.xlsx


---

## MODULE 1: TENANTS & CITY MODULES

**What it does:** Uploads state and city information + enabled modules.

**Excel Sheets:** Tenants, City_Modules

**Run this first!**

In [7]:
# Load Tenants and City Modules
print("="*60)
print("[MODULE 1] LOADING TENANTS & CITY MODULES")
print("="*60)

# Read data from Excel
tenants_data = reader.read_tenants()
city_modules_data = reader.read_city_modules()

print(f"\n[INFO] Loaded {len(tenants_data)} tenants from Excel")
for tenant in tenants_data:
    tenant_type = tenant.get('type', 'N/A')
    print(f"   - {tenant['code']}: {tenant['name']} ({tenant_type})")

print(f"\n[INFO] Loaded {len(city_modules_data)} city modules")

# Upload Tenants
result_tenants = uploader.create_mdms_data(
    schema_code='tenant.tenants',
    data_list=clean_nans(tenants_data),
    tenant=CONFIG['tenant_id']
)

# Upload City Modules
result_modules = uploader.create_mdms_data(
    schema_code='tenant.citymodule',
    data_list=clean_nans(city_modules_data),
    tenant=CONFIG['tenant_id']
)

# Check results
if result_tenants['failed'] == 0 and result_modules['failed'] == 0:
    print("\n[SUCCESS] Tenants & City Modules loaded successfully! Proceed to next module.")
else:
    print("\n[WARNING] Some items failed. Fix errors in Excel and re-run this cell.")

[MODULE 1] LOADING TENANTS & CITY MODULES

[INFO] Loaded 2 tenants from Excel
   - pg: Punjab State (State)
   - pg.citya: Punjab State (State)

[INFO] Loaded 3 city modules

[UPLOADING] tenant.tenants
   Tenant: pg
   Records: 2
   API URL: http://localhost:8094/mdms-v2/v2/_create/{schema_code}
   [OK] [1/2] pg
   [OK] [2/2] pg.citya
[SUMMARY] Created: 2
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 0

[UPLOADING] tenant.citymodule
   Tenant: pg
   Records: 3
   API URL: http://localhost:8094/mdms-v2/v2/_create/{schema_code}
   [OK] [1/3] PGR
   [OK] [2/3] HRMS
   [OK] [3/3] Workbench
[SUMMARY] Created: 3
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 0

[SUCCESS] Tenants & City Modules loaded successfully! Proceed to next module.


---

## MODULE 2: BOUNDARIES

**What it does:** Uploads geographic boundaries (City → Zone → Ward → Block → Locality).

**Excel Sheets:** Hierarchy_Definition, Boundary_Entities, Boundary_Relationships

**Prerequisites:** Tenants must be loaded first.

In [8]:
# Load Boundaries
print("="*60)
print("[MODULE 2] LOADING BOUNDARIES")
print("="*60)

# Read data from Excel
boundary_hierarchy = reader.read_boundary_hierarchy()
boundary_entities = reader.read_boundary_entities()
boundary_relationships = reader.read_boundary_relationships()

# Step 1: Upload Entities
print("\n[1/3] Uploading Boundary Entities...")
if boundary_entities:
    print(f"   Found {len(boundary_entities)} boundary entities")
    try:
        uploader.create_boundary_entities(boundary_entities)
    except Exception as e:
        print(f"   Note: {str(e)[:100]}")

# Step 2: Upload Hierarchy
print("\n[2/3] Uploading Boundary Hierarchy...")
if boundary_hierarchy:
    try:
        uploader.create_boundary_hierarchy(boundary_hierarchy)
    except Exception as e:
        print(f"   Note: {str(e)[:100]}")

# Step 3: Upload Relationships
print("\n[3/3] Uploading Boundary Relationships...")
if boundary_relationships:
    print(f"   Found {len(boundary_relationships)} relationships")
    created = 0
    failed = 0
    for i, rel in enumerate(boundary_relationships, 1):
        code = rel.get('code', 'Unknown')
        print(rel)
        try:
            uploader.create_boundary_relationship(rel)
            print(f"   [OK] [{i}/{len(boundary_relationships)}] {code}")
            created += 1
        except Exception as e:
            print(f"   [FAILED] [{i}/{len(boundary_relationships)}] {code}")
            print(f"   ERROR: {str(e)[:200]}")
            failed += 1
        
   
    
    print("="*60)
    print(f"[SUMMARY] Created: {created}, Failed: {failed}")
    print("="*60)

print("\n[SUCCESS] Boundaries loading complete. Check messages above for any errors.")

[MODULE 2] LOADING BOUNDARIES

[1/3] Uploading Boundary Entities...
   Found 9 boundary entities
   [SUCCESS] 9 boundary entities created

[2/3] Uploading Boundary Hierarchy...
   [SUCCESS] Boundary hierarchy created

[3/3] Uploading Boundary Relationships...
   Found 5 relationships
{'tenantId': 'pg.citya', 'code': 'CITYA', 'hierarchyType': 'ADMIN', 'boundaryType': 'City', 'parent': ''}
   [FAILED] [1/5] CITYA
   ERROR: HTTP 400: {"ResponseInfo":null,"Errors":[{"id":null,"parentId":null,"code":"BOUNDARY_ENTITY_DOES_NOT_EXIST","message":"Boundary entity does not exist.","description":null,"params":null}]}
{'tenantId': 'pg.citya', 'code': 'PG_CITYA_ADMIN_CITY', 'hierarchyType': 'ADMIN', 'boundaryType': 'Zone', 'parent': 'CITYA'}
   [FAILED] [2/5] PG_CITYA_ADMIN_CITY
   ERROR: HTTP 400: {"ResponseInfo":null,"Errors":[{"id":null,"parentId":null,"code":"BOUNDARY_ENTITY_DOES_NOT_EXIST","message":"Boundary entity does not exist.","description":null,"params":null}]}
{'tenantId': 'pg.citya', '

---

## MODULE 3: DEPARTMENTS

**What it does:** Uploads government departments.

**Excel Sheet:** Departments

**Prerequisites:** Boundaries must be loaded first.

**Example:** Street Lights, Building & Roads, Health & Sanitation

In [9]:
# Load Departments
print("="*60)
print("[MODULE 3] LOADING DEPARTMENTS")
print("="*60)

# Read data from Excel
departments_data = reader.read_departments()
print(departments_data)

print(f"\n[INFO] Loaded {len(departments_data)} departments from Excel")
for dept in departments_data[:5]:
    print(f"   - {dept['code']}: {dept['name']}")
if len(departments_data) > 5:
    print(f"   ... and {len(departments_data) - 5} more")

# Upload Departments
result = uploader.create_mdms_data(
    schema_code='common-masters.Department',
    data_list=clean_nans(departments_data),
    tenant=CONFIG['target_tenant_id']
)

# Check result
if result['failed'] == 0:
    print("\n[SUCCESS] Departments loaded successfully! Proceed to next module.")
else:
    print("\n[WARNING] Some departments failed. Fix errors in Excel and re-run this cell.")

[MODULE 3] LOADING DEPARTMENTS
[{'code': 'DEPT_1', 'name': 'Street Lights', 'active': True}, {'code': 'DEPT_2', 'name': 'Building & Roads', 'active': True}, {'code': 'DEPT_3', 'name': 'Health & Sanitation', 'active': True}]

[INFO] Loaded 3 departments from Excel
   - DEPT_1: Street Lights
   - DEPT_2: Building & Roads
   - DEPT_3: Health & Sanitation

[UPLOADING] common-masters.Department
   Tenant: pg.citya
   Records: 3
   API URL: http://localhost:8094/mdms-v2/v2/_create/{schema_code}
   [OK] [1/3] DEPT_1
   [OK] [2/3] DEPT_2
   [OK] [3/3] DEPT_3
[SUMMARY] Created: 3
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 0

[SUCCESS] Departments loaded successfully! Proceed to next module.


---

## MODULE 4: DESIGNATIONS

**What it does:** Uploads job titles/positions.

**Excel Sheet:** Designations

**Prerequisites:** Departments must be loaded first.

**Example:** Junior Engineer, Senior Engineer, Medical Officer

In [10]:
# Load Designations
print("="*60)
print("[MODULE 4] LOADING DESIGNATIONS")
print("="*60)

# Read data from Excel
designations_data = reader.read_designations()
print (designations_data)

print(f"\n[INFO] Loaded {len(designations_data)} designations from Excel")
for desig in designations_data[:5]:
    print(f"   - {desig['code']}: {desig['name']}")
if len(designations_data) > 5:
    print(f"   ... and {len(designations_data) - 5} more")

# Upload Designations
result = uploader.create_mdms_data(
    schema_code='common-masters.Designation',
    data_list=clean_nans(designations_data),
    tenant=CONFIG['target_tenant_id']
)

# Check result
if result['failed'] == 0:
    print("\n[SUCCESS] Designations loaded successfully! Proceed to next module.")
else:
    print("\n[WARNING] Some designations failed. Fix errors in Excel and re-run this cell.")

[MODULE 4] LOADING DESIGNATIONS
[{'code': 'DESIG_01', 'name': 'Superintending Engineer', 'departmentCode': 'DEPT_2', 'active': True, 'description': 'Senior Engineering Role'}]

[INFO] Loaded 1 designations from Excel
   - DESIG_01: Superintending Engineer

[UPLOADING] common-masters.Designation
   Tenant: pg.citya
   Records: 1
   API URL: http://localhost:8094/mdms-v2/v2/_create/{schema_code}
   [OK] [1/1] DESIG_01
[SUMMARY] Created: 1
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 0

[SUCCESS] Designations loaded successfully! Proceed to next module.


---

## MODULE 5: COMPLAINT TYPES

**What it does:** Uploads PGR complaint/service types.

**Excel Sheet:** ComplaintTypes

**Example:** Streetlight not working, Garbage collection, Road pothole

In [11]:
# Load Complaint Types
print("="*60)
print("[MODULE 5] LOADING COMPLAINT TYPES")
print("="*60)

# Read data from Excel
complaint_types_data = reader.read_complaint_types()
print(complaint_types_data)

print(f"\n[INFO] Loaded {len(complaint_types_data)} complaint types from Excel")
for ct in complaint_types_data[:5]:
    print(f"   - {ct['serviceCode']}: {ct['name']}")
if len(complaint_types_data) > 5:
    print(f"   ... and {len(complaint_types_data) - 5} more")

# Upload Complaint Types
result = uploader.create_mdms_data(
    schema_code='RAINMAKER-PGR.ServiceDefs',
    data_list=clean_nans(complaint_types_data),
    tenant=CONFIG['target_tenant_id']
)

# Check result
if result['failed'] == 0:
    print("\n[SUCCESS] Complaint Types loaded successfully! Proceed to next module.")
else:
    print("\n[WARNING] Some complaint types failed. Fix errors in Excel and re-run this cell.")

[MODULE 5] LOADING COMPLAINT TYPES
[{'serviceCode': 'StreetlightNotWorking', 'name': 'Streetlight not working', 'menuPath': 'Street Lights', 'active': True, 'department': 'DEPT_1', 'slaHours': 336, 'priority': 1}]

[INFO] Loaded 1 complaint types from Excel
   - StreetlightNotWorking: Streetlight not working

[UPLOADING] RAINMAKER-PGR.ServiceDefs
   Tenant: pg.citya
   Records: 1
   API URL: http://localhost:8094/mdms-v2/v2/_create/{schema_code}
   [OK] [1/1] StreetlightNotWorking
[SUMMARY] Created: 1
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 0

[SUCCESS] Complaint Types loaded successfully! Proceed to next module.


---

## MODULE 8: LOCALIZATION 


**What it does:** Uploads language translations.

**Excel Sheet:** Localization

**Note:** Skip this if you only need English.

In [12]:
# Load Localization 
print("="*60)
print("[MODULE 8] LOADING LOCALIZATION ")
print("="*60)

try:
    # Read localization data
    localization_data = reader.read_localization()
    
    if not localization_data:
        print("\n[INFO] No localization data found ")
    else:
        print(f"\n[INFO] Loaded {len(localization_data)} translations")
        
        # Upload Localization using dedicated localization API
        result = uploader.create_localization_messages(
            localization_list=clean_nans(localization_data),
            tenant=CONFIG['tenant_id']
        )
        
        if result['failed'] == 0:
            print("\n[SUCCESS] Localization loaded successfully!")
        else:
            print(f"\n[WARNING] Some localizations failed. Check errors above.")
            
except Exception as e:
    print(f"\n[INFO] Localization skipped: {str(e)}")

[MODULE 8] LOADING LOCALIZATION 

[INFO] Localization skipped: Worksheet named 'localization' not found
